In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
#from torch_geometric.nn import GCNConv
from torch_geometric.nn import GCNConv, GNNExplainer
#from GNN_explainer_modified import *

#This notebook is to test the pre-trained model with the test set.
import torch
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd

import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

from scipy import stats
from sklearn import metrics

from my_dataset_1by1 import *

import os
import time

import numpy as np


In [2]:
class GCN_regression_Net_single(torch.nn.Module):
    """
    feed the data one-by-one version.
    """
    def __init__(self, n_features, nhid1, nhid2, nhid3):
        super(GCN_regression_Net_single, self).__init__()
        self.conv1 = GCNConv(n_features, nhid1)
        self.conv2 = GCNConv(nhid1, nhid2)
        self.conv3 = GCNConv(nhid2, nhid3)
        #self.bn = torch.nn.BatchNorm1d(n_nodes)
        #self.dropout = dropout
        #self.linear1 = torch.nn.Linear(nhid3, 1)    #used in previous patch, reduce nhid first
        self.linear1 = torch.nn.Linear(nhid3, 1)
        self.linear2 = torch.nn.Linear(508, 1)
        #self.batch_size = batch_size
        #self.maxpool = torch.nn.MaxPool2d(4, stride = 4) # max pooling with square window of size=3, stride=2
        #no maxpooling because it will reduce the feature channel.
        
    def forward(self, x, edge_index, edge_weight):
        x, edge_index, edge_weight = x, edge_index, edge_weight #X has shape[batch_size * n_node]

        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        
        x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)
        
        x = self.conv3(x, edge_index, edge_weight)    # X in shape: [n_nodes * batch_size, nhid3]
        x = F.relu(x)
        
        x = F.dropout(x, training=self.training) 
        
        x = self.linear1(x) #output shape in [508,1]  
        
        #print("the output layer shape is:")
        #print(x.shape)
        
        
        x = torch.squeeze(x)    #output shape in [508]
        #print(type(x))
        x = self.linear2(x) #output shape in [1]
        x = torch.squeeze(x)
        print(type(x))
        #x = torch.reshape(x, (self.batch_size, -1)) # output shape in [batch_size, n_node]
        return x

In [3]:
def give_me_example(num):
    with torch.no_grad():
        datapoint = My_dataset().__getitem__(num)
        print("True y is: %f" %(datapoint.y))

        x, edge_index, edge_weight = datapoint.X, datapoint.edge_index, datapoint.edge_attr
        x, edge_index, edge_weight = Variable(torch.as_tensor(X, dtype=torch.float32)).to(device), edge_index.to(device), edge_weight.to(device)  
        prediction = model(x, edge_index, edge_weight)
    print("Prediction is: %f" %(prediction))
    return
    



In [4]:
np.random.seed(1)
torch.random.manual_seed(1)
device = torch.device('cuda')

dataset_size = 3000
perm = torch.randperm(dataset_size).numpy()
partition = {}
partition["train"] = perm[:int(dataset_size*8/10)]
partition["validation"] = perm[int(dataset_size*8/10):int(dataset_size*9/10)]
partition["test"] = perm[int(dataset_size*9/10):]

model = GCN_regression_Net_single(n_features = 6, 
                               nhid1 = 32, 
                               nhid2 = 64, 
                               nhid3 = 128).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1E-6, weight_decay=5e-4)
loss_func = torch.nn.MSELoss()


earlystopping = True
best_epoch_num = 0
best_loss_valid = 1e10
patient = 100
prefix = "gcn_net_trained_18_single_MSELoss"
max_epochs = 500
    
train_set = torch.utils.data.Subset(My_dataset(), partition["train"])
validation_set = torch.utils.data.Subset(My_dataset(), partition["validation"])
test_set = torch.utils.data.Subset(My_dataset(), partition["test"])

In [5]:
next(model.parameters()).is_cuda

True

In [9]:
PATH = 'gcn_net_trained_11_16batched_MSELoss.pt.final'
model.load_state_dict(torch.load(PATH))

FileNotFoundError: [Errno 2] No such file or directory: 'gcn_net_trained_11_16batched_MSELoss.pt.final'

In [7]:
explainer = GNNExplainer(model, epochs=200)

datapoint = next(iter(train_set))
x, edge_index, edge_weight, y = datapoint.X, datapoint.edge_index, datapoint.edge_attr, datapoint.y
x, edge_index, edge_weight = Variable(torch.as_tensor(x, dtype=torch.float32)).to(device), edge_index.to(device), edge_weight.to(device)
y = Variable(torch.as_tensor(y, dtype=torch.float32)).to(device)
            
for item in (x, edge_index, edge_weight, y):
    print(item.type)
    print(item.shape)

print(y.item())

<built-in method type of Tensor object at 0x0000029ADC028840>
torch.Size([508, 6])
<built-in method type of Tensor object at 0x0000029ADC028CC0>
torch.Size([2, 41258])
<built-in method type of Tensor object at 0x0000029ADAC84600>
torch.Size([41258])
<built-in method type of Tensor object at 0x0000029ADC03FFC0>
torch.Size([])
910.0


In [8]:
node_idx = 100
node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edge_index, edge_weight = edge_weight)
ax, G = explainer.visualize_subgraph(node_idx, edge_index, edge_mask, y=y.item())
plt.show()

Explain node 100:   0%|                                                                        | 0/200 [00:00<?, ?it/s]

<class 'torch.Tensor'>
<class 'torch.Tensor'>


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [ ]:
node_idx = -1
gnode_feat_mask, gedge_mask = explainer.explain_graph(x, edge_index, edge_weight = edge_weight)
ax, G = explainer.visualize_subgraph(node_idx, edge_index, gedge_mask, y=data.y)
plt.show()